# Air Quality in Winnipeg

In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import sklearn
import matplotlib as plt
import seaborn as sns
import folium


Read in the Air Quality dataset which was downloaded from the city open data portal.

In [2]:
df = pd.read_csv("D:/DATA/City_WPG/Air_quality/Air_Quality.csv")

C:\Users\ZAP-DS\Anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237636 entries, 0 to 2237635
Data columns (total 8 columns):
ObservationID       int64
ObservationTime     object
ThingID             int64
LocationName        object
MeasurementType     object
MeasurementValue    object
MeasurementUnit     object
location            object
dtypes: int64(2), object(6)
memory usage: 136.6+ MB


In [4]:
df.describe()

,ObservationID,ThingID
count,2.237636e+06,2.237636e+06
mean,1.139608e+06,9.457337e+01
std,6.631953e+05,1.883241e+01
min,1.270000e+02,6.600000e+01
25%,5.595458e+05,7.800000e+01
50%,1.158608e+06,9.600000e+01
75%,1.718017e+06,1.140000e+02
max,2.278633e+06,1.200000e+02


Let's take a look at the first 10 rows to get a look at the data.

In [5]:
df.head(10)

,ObservationID,ObservationTime,ThingID,LocationName,MeasurementType,MeasurementValue,MeasurementUnit,location
0,613550,12/08/2017 08:01:37 PM +0000,108,River East,Temperature,-1.44,C,"(49.91346, -97.070039)"
1,613551,12/08/2017 08:01:37 PM +0000,108,River East,Humidity,74.95,%,"(49.91346, -97.070039)"
2,613552,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,PM2.5 Particulates,6,ug/m3,"(49.859235, -97.13801)"
3,613553,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,Temperature,-99.99,C,"(49.859235, -97.13801)"
4,613554,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,Humidity,-99.99,%,"(49.859235, -97.13801)"
5,613555,12/08/2017 08:02:16 PM +0000,114,Point Douglas,PM2.5 Particulates,9,ug/m3,"(49.92334, -97.114238)"
6,613556,12/08/2017 08:02:16 PM +0000,114,Point Douglas,Temperature,-1.33,C,"(49.92334, -97.114238)"
7,613557,12/08/2017 08:02:16 PM +0000,114,Point Douglas,Humidity,73.6,%,"(49.92334, -97.114238)"
8,613558,12/08/2017 08:03:32 PM +0000,72,Fort Garry,PM2.5 Particulates,5,ug/m3,"(49.791663, -97.165666)"
9,613559,12/08/2017 08:03:32 PM +0000,72,Fort Garry,Temperature,-0.99,C,"(49.791663, -97.165666)"


In [6]:
df['ObservationTime'].head()

0    12/08/2017 08:01:37 PM +0000
1    12/08/2017 08:01:37 PM +0000
2    12/08/2017 08:01:37 PM +0000
3    12/08/2017 08:01:37 PM +0000
4    12/08/2017 08:01:37 PM +0000
Name: ObservationTime, dtype: object

In [7]:
df.tail()

,ObservationID,ObservationTime,ThingID,LocationName,MeasurementType,MeasurementValue,MeasurementUnit,location
2237631,2270234,02/13/2019 11:27:42 PM +0000,120,St. James East,PM2.5 Particulates,11,ug/m3,"(49.879133, -97.205769)"
2237632,2270235,02/13/2019 11:27:42 PM +0000,120,St. James East,Temperature,-11.27,C,"(49.879133, -97.205769)"
2237633,2270236,02/13/2019 11:27:42 PM +0000,120,St. James East,Humidity,75.17,%,"(49.879133, -97.205769)"
2237634,2270237,02/13/2019 11:29:08 PM +0000,108,River East,PM2.5 Particulates,14,ug/m3,"(49.91346, -97.070039)"
2237635,2272136,02/15/2019 02:04:42 AM +0000,120,St. James East,PM2.5 Particulates,1,ug/m3,"(49.879133, -97.205769)"


Now a shorter version of the dataset is going to be created for testing data cleaning and manipulation which will then be implemented on the larger dataset

In [8]:
short = df.iloc[:5]
short

,ObservationID,ObservationTime,ThingID,LocationName,MeasurementType,MeasurementValue,MeasurementUnit,location
0,613550,12/08/2017 08:01:37 PM +0000,108,River East,Temperature,-1.44,C,"(49.91346, -97.070039)"
1,613551,12/08/2017 08:01:37 PM +0000,108,River East,Humidity,74.95,%,"(49.91346, -97.070039)"
2,613552,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,PM2.5 Particulates,6,ug/m3,"(49.859235, -97.13801)"
3,613553,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,Temperature,-99.99,C,"(49.859235, -97.13801)"
4,613554,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,Humidity,-99.99,%,"(49.859235, -97.13801)"


The folium package is going to be used for plotting 

In [10]:
short['lat'] = short.location.str.strip('()').str.split(', ').str[0]
short['long'] = short.location.str.strip('()').str.split(', ').str[1]
print(type(short))
short

<class 'pandas.core.frame.DataFrame'>


C:\Users\ZAP-DS\Anaconda3\envs\tf15\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\ZAP-DS\Anaconda3\envs\tf15\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ObservationID,ObservationTime,ThingID,LocationName,MeasurementType,MeasurementValue,MeasurementUnit,location,lat,long
0,613550,12/08/2017 08:01:37 PM +0000,108,River East,Temperature,-1.44,C,"(49.91346, -97.070039)",49.91346,-97.070039
1,613551,12/08/2017 08:01:37 PM +0000,108,River East,Humidity,74.95,%,"(49.91346, -97.070039)",49.91346,-97.070039
2,613552,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,PM2.5 Particulates,6,ug/m3,"(49.859235, -97.13801)",49.859235,-97.13801
3,613553,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,Temperature,-99.99,C,"(49.859235, -97.13801)",49.859235,-97.13801
4,613554,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,Humidity,-99.99,%,"(49.859235, -97.13801)",49.859235,-97.13801


In [11]:
df['lat'] = df.location.str.strip('()').str.split(', ').str[0]
df['long'] = df.location.str.strip('()').str.split(', ').str[1]
print(type(df))
df.head()

<class 'pandas.core.frame.DataFrame'>


,ObservationID,ObservationTime,ThingID,LocationName,MeasurementType,MeasurementValue,MeasurementUnit,location,lat,long
0,613550,12/08/2017 08:01:37 PM +0000,108,River East,Temperature,-1.44,C,"(49.91346, -97.070039)",49.91346,-97.070039
1,613551,12/08/2017 08:01:37 PM +0000,108,River East,Humidity,74.95,%,"(49.91346, -97.070039)",49.91346,-97.070039
2,613552,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,PM2.5 Particulates,6,ug/m3,"(49.859235, -97.13801)",49.859235,-97.13801
3,613553,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,Temperature,-99.99,C,"(49.859235, -97.13801)",49.859235,-97.13801
4,613554,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,Humidity,-99.99,%,"(49.859235, -97.13801)",49.859235,-97.13801


Time to isolate the different measuremtn types within the data itself. The measurement types recorded are Temperature, Humidity, and Particulates.

In [12]:
df_temp = df[(df['MeasurementType'] == 'Temperature')]

In [13]:
df_humid = df[df['MeasurementType'] == 'Humidity']

In [14]:
df_particles = df[df['MeasurementType'] == 'PM2.5 Particulates']

Let's check out the first 5 values of each of the recorded measurements to make sure it worked out.

In [15]:
df_temp.head()

,ObservationID,ObservationTime,ThingID,LocationName,MeasurementType,MeasurementValue,MeasurementUnit,location,lat,long
0,613550,12/08/2017 08:01:37 PM +0000,108,River East,Temperature,-1.44,C,"(49.91346, -97.070039)",49.91346,-97.070039
3,613553,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,Temperature,-99.99,C,"(49.859235, -97.13801)",49.859235,-97.13801
6,613556,12/08/2017 08:02:16 PM +0000,114,Point Douglas,Temperature,-1.33,C,"(49.92334, -97.114238)",49.92334,-97.114238
9,613559,12/08/2017 08:03:32 PM +0000,72,Fort Garry,Temperature,-0.99,C,"(49.791663, -97.165666)",49.791663,-97.165666
12,613562,12/08/2017 08:03:43 PM +0000,78,St. Boniface #1,Temperature,-1.13,C,"(49.876126, -97.108649)",49.876126,-97.108649


In [16]:
df_humid.head()

,ObservationID,ObservationTime,ThingID,LocationName,MeasurementType,MeasurementValue,MeasurementUnit,location,lat,long
1,613551,12/08/2017 08:01:37 PM +0000,108,River East,Humidity,74.95,%,"(49.91346, -97.070039)",49.91346,-97.070039
4,613554,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,Humidity,-99.99,%,"(49.859235, -97.13801)",49.859235,-97.13801
7,613557,12/08/2017 08:02:16 PM +0000,114,Point Douglas,Humidity,73.6,%,"(49.92334, -97.114238)",49.92334,-97.114238
10,613560,12/08/2017 08:03:32 PM +0000,72,Fort Garry,Humidity,72.34,%,"(49.791663, -97.165666)",49.791663,-97.165666
13,613563,12/08/2017 08:03:43 PM +0000,78,St. Boniface #1,Humidity,71.68,%,"(49.876126, -97.108649)",49.876126,-97.108649


In [17]:
df_particles.head()

,ObservationID,ObservationTime,ThingID,LocationName,MeasurementType,MeasurementValue,MeasurementUnit,location,lat,long
2,613552,12/08/2017 08:01:37 PM +0000,84,Fort Rouge,PM2.5 Particulates,6,ug/m3,"(49.859235, -97.13801)",49.859235,-97.13801
5,613555,12/08/2017 08:02:16 PM +0000,114,Point Douglas,PM2.5 Particulates,9,ug/m3,"(49.92334, -97.114238)",49.92334,-97.114238
8,613558,12/08/2017 08:03:32 PM +0000,72,Fort Garry,PM2.5 Particulates,5,ug/m3,"(49.791663, -97.165666)",49.791663,-97.165666
11,613561,12/08/2017 08:03:42 PM +0000,78,St. Boniface #1,PM2.5 Particulates,7,ug/m3,"(49.876126, -97.108649)",49.876126,-97.108649
14,613564,12/08/2017 08:05:47 PM +0000,96,Transcona #2,PM2.5 Particulates,6,ug/m3,"(49.905908, -96.975043)",49.905908,-96.975043


Looking briefly at the df_temp and df_humid we can see that one of the readings doesn't make sense either from malfunctioning sensors or data recording error.

## Time to clean up the data

Knowing that an error produces -99.99 as a result, all readings of this value must be removed. 

In [18]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 747876 entries, 0 to 2237632
Data columns (total 10 columns):
ObservationID       747876 non-null int64
ObservationTime     747876 non-null object
ThingID             747876 non-null int64
LocationName        747876 non-null object
MeasurementType     747876 non-null object
MeasurementValue    747876 non-null object
MeasurementUnit     747876 non-null object
location            747876 non-null object
lat                 747876 non-null object
long                747876 non-null object
dtypes: int64(2), object(8)
memory usage: 62.8+ MB


In [19]:
df_temp['MeasurementValue'] = df_temp['MeasurementValue'].str.replace('-99.99', '')

C:\Users\ZAP-DS\Anaconda3\envs\tf15\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


A list of the unique latitudinal and logitudinal values of the sampling locations was created using the .unique() method. To plot the locations the latitude and longitude values need to be changed from dtype object to float using .astype()

In [20]:
local_long = df.long.unique().astype(float)
print(local_long)

[-97.070039 -97.13801  -97.114238 -97.165666 -97.108649 -96.975043
 -97.205769 -97.080376 -97.215121 -97.003314]


In [21]:
local_lat = df.lat.unique().astype(float)
print(local_lat)

[49.91346  49.859235 49.92334  49.791663 49.876126 49.905908 49.879133
 49.838129 49.936131 49.901548]


Using the .unique() method again to capture the location names

In [22]:
local_name = df.LocationName.unique()
print(local_name)

['River East' 'Fort Rouge' 'Point Douglas' 'Fort Garry' 'St. Boniface #1'
 'Transcona #2' 'St. James East' 'St. Boniface #2' 'Tyndall'
 'Transcona #1']


In [23]:
locations = pd.DataFrame({"Location Name":local_name, "Latitude":local_lat, "Longitude":local_long})
locations

,Location Name,Latitude,Longitude
0,River East,49.913460,-97.070039
1,Fort Rouge,49.859235,-97.138010
2,Point Douglas,49.923340,-97.114238
3,Fort Garry,49.791663,-97.165666
4,St. Boniface #1,49.876126,-97.108649
5,Transcona #2,49.905908,-96.975043
6,St. James East,49.879133,-97.205769
7,St. Boniface #2,49.838129,-97.080376
8,Tyndall,49.936131,-97.215121
9,Transcona #1,49.901548,-97.003314


I just want to see how many readings there are for each location

In [24]:
spot = df.groupby(["LocationName"]).count()
print(spot)

                 ObservationID  ObservationTime  ThingID  MeasurementType  \
LocationName                                                                
Fort Garry              218504           218504   218504           218504   
Fort Rouge              264329           264329   264329           264329   
Point Douglas           130551           130551   130551           130551   
River East              316559           316559   316559           316559   
St. Boniface #1         290482           290482   290482           290482   
St. Boniface #2         211024           211024   211024           211024   
St. James East          451543           451543   451543           451543   
Transcona #1             47735            47735    47735            47735   
Transcona #2            292922           292922   292922           292922   
Tyndall                  13987            13987    13987            13987   

                 MeasurementValue  MeasurementUnit  location     lat    lon

Folium created map showing the location of the sampling sites

In [25]:
# create map of Winnipeg locations using latitude and longitude values
map_wpg = folium.Map(location=[locations['Latitude'].mean(), locations['Longitude'].mean()], zoom_start=11)

# add markers to map
for lat, lng, Location in zip(locations['Latitude'], locations['Longitude'], locations['Location Name']):
    label = '{}'.format(Location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_wpg)  
    
map_wpg